In [1]:
from __future__ import absolute_import
import aiida

In [2]:
from aiida import load_profile
load_profile()

# Import commonly used functionality
import numpy as np
from aiida import orm, engine, common
from aiida.plugins import WorkflowFactory
from aiida.orm import Code



In [3]:
#!pwd

In [4]:
!reentry scan

In [5]:
!verdi daemon restart
#!verdi daemon restart
#!$JUPYTER_PATH
#!export JUPYTER_PATH="/home/aakhtar/Projects/aiida-defects-siesta/siesta-defect-formation/aiida-workflow/formation_energy/;$JUPYTER_PATH"

Restarting the daemon... OK


In [6]:
from aiida_siesta.workflows.defect_formation.formation_energy_siesta import FormationEnergyWorkchainSIESTA
#import .formation_energy_siesta
#!pwd

In [7]:
from aiida.orm import StructureData
#Structure Pure
cell = [[15.0, 0.0, 0.0,],
        [ 0.0,15.0, 0.0,],
        [ 0.0, 0.0,15.0,],
        ]
pure = StructureData(cell=cell)
pure.append_atom(position=( 0.000 , 0.000 , 0.000 ),symbols=['O']) #1
#pure.append_atom(position=( 0.757 , 0.586 , 0.000 ),symbols=['H']) #2
#pure.append_atom(position=(-0.757 , 0.586 , 0.000),symbols=['H']) #3 
#pure.append_atom(position=( 0.000 , 3.500 , 0.000),symbols=['O']) #4
#pure.append_atom(position=( 0.757 , 2.914 , 0.000 ),symbols=['H']) #5
#pure.append_atom(position=(-0.757 , 2.914 , 0.000),symbols=['H']) #6


defect=StructureData(cell=cell)
defect.append_atom(position=( 0.000 , 0.000 , 0.000 ),symbols=['O']) #1
#defect.append_atom(position=( 0.757 , 0.586 , 0.000 ),symbols=['H']) #2
#defect.append_atom(position=(-0.757 , 0.586 , 0.000),symbols=['H']) #3 
#defect.append_atom(position=( 0.000 , 3.500 , 0.000),symbols=['O']) #4
#defect.append_atom(position=( 0.757 , 2.914 , 0.000 ),symbols=['H']) #5
#defect.append_atom(position=(-0.757 , 2.914 , 0.000),symbols=['H'],name="GhostH") #6

In [8]:
code = Code.get_from_string('siesta-psml-lua@N552VW')
charge=-2

In [9]:
from aiida.orm import Dict
parameters_host = Dict(dict={
   "mesh-cutoff": "250 Ry",
   "dm-tolerance": "0.0001",
   "MD-TypeOfRun":   "cg",
    "MD-NumCGsteps": "5000",
   #"LUA-script":   "neb.lua",
   "DM-NumberPulay ":  "3",
   "DM-History-Depth":  "0",
   "SCF-Mixer-weight":  "0.02",
   "SCF-Mix":   "density",
   "SCF-Mixer-kick":  "35",
   "MD-VariableCell":  "F",
   "MD-MaxCGDispl":  "0.3 Bohr",
   "MD-MaxForceTol":  " 0.04000 eV/Ang", 
    })
parameters_defect_q0 = Dict(dict={
   "mesh-cutoff": "250 Ry",
   "dm-tolerance": "0.0001",
   "MD-TypeOfRun":   "cg",
    "MD-NumCGsteps": "5000",
   #"LUA-script":   "neb.lua",
   "DM-NumberPulay ":  "3",
   "DM-History-Depth":  "0",
   "SCF-Mixer-weight":  "0.02",
   "SCF-Mix":   "density",
   "SCF-Mixer-kick":  "35",
   "MD-VariableCell":  "F",
   "MD-MaxCGDispl":  "0.3 Bohr",
   "MD-MaxForceTol":  " 0.04000 eV/Ang", 
   "NetCharge": "0",
    })
parameters_defect_q = Dict(dict={
   "mesh-cutoff": "250 Ry",
   "dm-tolerance": "0.0001",
   "MD-TypeOfRun":   "cg",
    "MD-NumCGsteps": "5000",
   #"LUA-script":   "neb.lua",
   "DM-NumberPulay ":  "3",
   "DM-History-Depth":  "0",
   "SCF-Mixer-weight":  "0.02",
   "SCF-Mix":   "density",
   "SCF-Mixer-kick":  "35",
   "MD-VariableCell":  "F",
   "MD-MaxCGDispl":  "0.3 Bohr",
   "MD-MaxForceTol":  " 0.04000 eV/Ang",
   "NetCharge": str(charge),
    })
#options_host=Dict(dict={'options':{"max_wallclock_seconds": 360},
#                        "resources":{"num_machines": 1,
#                                      "num_mpiprocs_per_machine": 1}
#                        })
options_host=Dict(dict={
        "max_wallclock_seconds": 360000,
        #'withmpi': True,
        #'account': "tcphy113c",
        #'queue_name': "DevQ",
        "resources": {
            "num_machines": 1,
            "num_mpiprocs_per_machine": 1,
        }
    })
options_defect_q0=Dict(dict={
        "max_wallclock_seconds": 360000,
        #'withmpi': True,
        #'account': "tcphy113c",
        #'queue_name': "DevQ",
        "resources": {
            "num_machines": 1,
            "num_mpiprocs_per_machine": 1,
        }
    })
options_defect_q=Dict(dict={
        "max_wallclock_seconds": 360000,
        #'withmpi': True,
        #'account': "tcphy113c",
        #'queue_name': "DevQ",
        "resources": {
            "num_machines": 1,
            "num_mpiprocs_per_machine": 1,
        }
    })



In [10]:
basis_dict_host =Dict(dict= {
'pao-basistype':'split',
'pao-splitnorm': 0.150,
'pao-energyshift': '0.020 Ry',
'%block pao-basis-sizes':
"""
O DZP
#H    DZP
%endblock pao-basis-sizes""",
})
basis_dict_defect_q0 = Dict(dict= {
'pao-basistype':'split',
'pao-splitnorm': 0.150,
'pao-energyshift': '0.020 Ry',
'%block pao-basis-sizes':
"""
O DZP
#GhostH DZP
#H    DZP
%endblock pao-basis-sizes""",
})
basis_dict_defect_q =Dict(dict= {
'pao-basistype':'split',
'pao-splitnorm': 0.150,
'pao-energyshift': '0.020 Ry',
'%block pao-basis-sizes':
"""
O DZP
#GhostH DZP
#H    DZP
%endblock pao-basis-sizes""",
})

In [11]:
parameters_defect_q0.get_dict()

{'mesh-cutoff': '250 Ry',
 'dm-tolerance': '0.0001',
 'MD-TypeOfRun': 'cg',
 'MD-NumCGsteps': '5000',
 'DM-NumberPulay ': '3',
 'DM-History-Depth': '0',
 'SCF-Mixer-weight': '0.02',
 'SCF-Mix': 'density',
 'SCF-Mixer-kick': '35',
 'MD-VariableCell': 'F',
 'MD-MaxCGDispl': '0.3 Bohr',
 'MD-MaxForceTol': ' 0.04000 eV/Ang',
 'NetCharge': '0'}

In [12]:
kpoints_host = orm.KpointsData()
kpoints_host.set_kpoints_mesh([1,1,1]) # Definately not converged, but we want the example to run quickly
kpoints_defect_q0 = orm.KpointsData()
kpoints_defect_q0.set_kpoints_mesh([1,1,1]) # Definately not converged, but we want the example to run quickly
kpoints_defect_q = orm.KpointsData()
kpoints_defect_q.set_kpoints_mesh([1,1,1]) # Definately not converged, but we want the example to run quickly

In [13]:
import os
from aiida_siesta.data.psf import PsfData
pseudo_file_to_species_map = [ ("O.psf", ['O'])]
#pseudo_file_to_species_map = [ ("H.psf", ['H']),("O.psf", ['O'])]
#pseudo_file_to_species_map = [ ("H.psf",["H"]),("GhostH.psf",["GhostH"]),("O.psf", ['O'])]
pseudos_dict_host = {}
for fname, kinds, in pseudo_file_to_species_map:
      absname = os.path.realpath(os.path.join("./pseudos",fname))
      pseudo, created = PsfData.get_or_create(absname, use_first=True)
      for j in kinds:
              pseudos_dict_host[j]=pseudo                
pseudos_dict_host


{'O': <PsfData: uuid: 4e885319-c33a-42fb-ae15-6f923cb84c5a (pk: 3)>}

In [14]:
pseudo_file_to_species_map = [("O.psf", ['O'])]
#pseudo_file_to_species_map = [ ("H.psf", ['GhostH','H']),("O.psf", ['O'])]
pseudos_dict_defect = {}
for fname, kinds, in pseudo_file_to_species_map:
      absname = os.path.realpath(os.path.join("./pseudos",fname))
      pseudo, created = PsfData.get_or_create(absname, use_first=True)
      for j in kinds:
              pseudos_dict_defect[j]=pseudo
pseudos_dict_defect

{'O': <PsfData: uuid: 4e885319-c33a-42fb-ae15-6f923cb84c5a (pk: 3)>}

In [15]:
inputs = {
    # Structures
    'host_structure': pure,
    'defect_structure': defect,
    # Defect information 
    #'defect_charge' : orm.Float(-2.),
    'defect_charge' : orm.Float(charge),
    'defect_site' : orm.List(list=[-0.757 , 2.914 , 0.000]),    # Position of the defect in crystal coordinates
    'fermi_level' : orm.Float(0.0),               # Position of the Fermi level, with respect to the valence band maximum      
    'chemical_potential' : orm.Float(250.709), # eV, the chemical potentical of a C atom
    'gaussian_sigma':orm.Float(0.5),
    'correction_scheme' : orm.Str('none'),
    "epsilon":orm.Float(1.0),
    "pseudos_host":pseudos_dict_host,
    "pseudos_defect":pseudos_dict_defect,
    #"pseudos_q0":pseudos_dict,
    #"pseudos_q":pseudos_dict,
    # Computational (chosen code is QE)
    'siesta' : { 'dft': {'supercell_host':{'code': code, 'kpoints': kpoints_host, 'parameters' : parameters_host,
                                           'options':options_host,"basis": basis_dict_host},
                         'supercell_defect_q0':{'code': code, 'kpoints': kpoints_defect_q,'parameters' : parameters_defect_q0,
                                                'options':options_defect_q0,"basis": basis_dict_defect_q0},
                         'supercell_defect_q':{'code': code, 'kpoints': kpoints_defect_q,'parameters' : parameters_defect_q,
                                               'options':options_defect_q,"basis": basis_dict_defect_q}
}}}

In [16]:
inputs

{'host_structure': <StructureData: uuid: eba91ed0-8ba1-46ab-b2ae-e532de9d9a42 (unstored)>,
 'defect_structure': <StructureData: uuid: 4eb487d7-98a2-422b-80db-cfddded372d8 (unstored)>,
 'defect_charge': <Float: uuid: 48817261-dcc6-4f8c-986d-459f0c4f89c2 (unstored) value: -2.0>,
 'defect_site': <List: uuid: 2659cd86-17b1-4581-9cee-2c6a38efacf4 (unstored) value: [-0.757, 2.914, 0.0]>,
 'fermi_level': <Float: uuid: c334b05b-4894-4eee-b802-e0932dee0736 (unstored) value: 0.0>,
 'chemical_potential': <Float: uuid: 146168a4-3915-4f47-a510-e8c8c33db028 (unstored) value: 250.709>,
 'gaussian_sigma': <Float: uuid: ac1d3347-41cf-4f4a-a0df-7cbeac0f6737 (unstored) value: 0.5>,
 'correction_scheme': <Str: uuid: f4d27173-04c5-4150-a38d-b94f75cc9936 (unstored) value: none>,
 'epsilon': <Float: uuid: 185afd76-cdd2-49ec-9418-5ea5832ae419 (unstored) value: 1.0>,
 'pseudos_host': {'O': <PsfData: uuid: 4e885319-c33a-42fb-ae15-6f923cb84c5a (pk: 3)>},
 'pseudos_defect': {'O': <PsfData: uuid: 4e885319-c33a-42f

In [17]:
workchain_future = engine.submit(FormationEnergyWorkchainSIESTA, **inputs)

/home/aakhtar/venv/aiida/lib/python3.6/site-packages/aiida/engine/processes/ports.py:111: UserWarning: default of input port `fermi_level` is a `Node` instance, which can lead to unexpected side effects. It is advised to use a lambda instead, e.g.: `default=lambda: orm.Int(5)`.
  warnings.warn(UserWarning(message))  # pylint: disable=no-member


In [18]:
workchain_future

<WorkChainNode: uuid: 24c9a196-77f2-4f18-a105-378bf00e105b (pk: 3160) (aiida.workflows:siesta.formation_energy_siesta)>

In [19]:
!verdi process list -a -p 1

  PK  Created    Process label                   Process State     Process status
----  ---------  ------------------------------  ----------------  ---------------------------------------------
2411  23h ago    FormationEnergyWorkchainSIESTA  ☠ Killed          Killed through `verdi process kill`
2412  23h ago    SiestaCalculation               ☠ Killed          Killed by parent<2411>
2413  23h ago    SiestaCalculation               ☠ Killed          Killed by parent<2411>
2414  23h ago    SiestaCalculation               ☠ Killed          Killed by parent<2411>
2438  23h ago    FormationEnergyWorkchainSIESTA  ☠ Killed          Killed through `verdi process kill`
2439  23h ago    SiestaCalculation               ☠ Killed          Killed by parent<2438>
2440  23h ago    SiestaCalculation               ☠ Killed          Killed by parent<2438>
2442  23h ago    SiestaCalculation               ☠ Killed          Killed by parent<2438>
2465  23h ago    FormationEnergyWorkchainSIESTA  ☠ Killed  

In [ ]:
#!verdi process kill 2669

In [20]:
!verdi process show 3160

Property     Value
-----------  ------------------------------------
type         FormationEnergyWorkchainSIESTA
state        Waiting
pk           3160
uuid         24c9a196-77f2-4f18-a105-378bf00e105b
label
description
ctime        2020-07-17 11:47:54.819648+00:00
mtime        2020-07-17 11:47:56.908830+00:00
computer     [1] N552VW

Inputs                       PK    Type
---------------------------  ----  -------------
pseudos_defect
    O                        3     PsfData
pseudos_host
    O                        3     PsfData
siesta
    dft
        supercell_defect_q
            basis            3159  Dict
            options          3158  Dict
            parameters       3157  Dict
            kpoints          3153  KpointsData
            code             403   Code
        supercell_defect_q0
            basis            3156  Dict
            options          3155  Dict
            parameters       3154  Dict
            kpoints          3153  KpointsData
            code

In [22]:
!verdi process report 3160

2020-07-17 11:47:56 [613 | REPORT]: [3160|FormationEnergyWorkchainSIESTA|setup]: Checking Formation Scheme
2020-07-17 11:47:56 [614 | REPORT]: [3160|FormationEnergyWorkchainSIESTA|run_dft_calcs]: Setting Up the No correction Formation Energy Workchain 
2020-07-17 11:47:56 [615 | REPORT]: [3160|FormationEnergyWorkchainSIESTA|run_dft_calcs]: Workflow Launching SIESTA for host structure (PK=3140)  (PK=3161)
2020-07-17 11:47:56 [616 | REPORT]: [3160|FormationEnergyWorkchainSIESTA|run_dft_calcs]: Launching SIESTA for defect structure (PK=3141) with charge 0.0 (PK=3162)
2020-07-17 11:47:56 [617 | REPORT]: [3160|FormationEnergyWorkchainSIESTA|run_dft_calcs]: Launching SIESTA for defect structure (PK=3141) with charge -2.0 (PK=3163)
2020-07-17 11:48:39 [618 | REPORT]: [3160|FormationEnergyWorkchainSIESTA|correction_required]: Ther will be no Corrections applied
2020-07-17 11:48:40 [619 | REPORT]: [3160|FormationEnergyWorkchainSIESTA|check_dft_calcs]: Checking Up Whether DFT Caclucations are Fi

In [ ]:
#workchain_future.outputs.formation_energy_uncorrected.value #.outputs.formation_energy_uncorrected.value
from aiida.orm import load_node
results=load_node('2952')

In [ ]:
results.outputs['output_parameters'].get_dict()#['Etot']
#results['forces_and_stress']

In [ ]:
results.outputs.output_parameters.get_dict()

In [ ]:
code

In [ ]:
from aiida_siesta.calculations.siesta import SiestaCalculation
builder = SiestaCalculation.get_builder()

In [ ]:
builder.metadata